### Goal of this program: To take inputs (stats from tennis players) and output predicted winning percentage

### Steps:
###### 1) Scrape data from tennis database (tennisabstract.com for now)
###### 2) input data into model
###### 3) Model ouputs predicted winning percentage (and maybe predicted outcome against opponent in the future)

In [18]:
import numpy as np
import random
from random import randint
from sklearn.preprocessing import MinMaxScaler
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPool2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import *
from keras.models import Model
from keras.applications import imagenet_utils
from keras.preprocessing import image
import matplotlib.pyplot as plt 
from sklearn.metrics import confusion_matrix
import itertools
%matplotlib inline

import os
import json
# import nbimporter


In [4]:
from serve_scraper import data

# import DataTransfer

ImportError: cannot import name 'data' from 'serve_scraper' (/Users/spencerweishaar/machineLearning/tennis-predictions/serve_scraper.py)

In [5]:
with open("data.json", "r") as file:
    data = json.load(file)

### Setting all the training/valid sets up 

In [ ]:
# Data dependent on match type (singles and doubles) will be put in corresponding lists. These corresponding lists
# will all be put together in another list. This list is then paired with another list in a dictionary. For example,
# singles training sample data will be put in their corresponding lists. These lists will be put in a singles training
# sample list, which countains all the lists with the singles training sample data. This singles training list will 
# then be paired with a doubles training sample list in a dictionary. 

In [37]:
training_sets = dict()
valid_sets = dict()

singlesUtr_train_samples = []
doublesUtr_train_samples = []
servePercentage_train_samples = []

singlesGameDiff_train_samples = []
singlesWeightedGameDiff_train_samples = []
singlesWinPercentage_train_samples = []
singlesOpponentUtr_train_samples = []
singlesLongestWinStreak_train_samples = []

training_sets["singles"] = [singlesGameDiff_train_samples, singlesWeightedGameDiff_train_samples, singlesWinPercentage_train_samples, singlesOpponentUtr_train_samples, singlesLongestWinStreak_train_samples]
# print(singles_train_lists)

doublesGameDiff_train_samples = []
doublesWeightedGameDiff_train_samples = []
doublesWinPercentage_train_samples = []
doublesOpponentUtr_train_samples = []
doublesLongestWinStreak_train_samples = []

training_sets["doubles"] = [doublesGameDiff_train_samples, doublesWeightedGameDiff_train_samples, doublesWinPercentage_train_samples, doublesOpponentUtr_train_samples, singlesLongestWinStreak_train_samples]


serveSpeed_train_labels = []



singlesUtr_valid_samples = []
doublesUtr_valid_samples = []
servePercentage_valid_samples = []

singlesGameDiff_valid_samples = []
singlesWeightedGameDiff_valid_samples = []
singlesWinPercentage_valid_samples = []
singlesOpponentUtr_valid_samples = []
singlesLongestWinStreak_valid_samples = []

valid_sets["singles"] = [singlesGameDiff_valid_samples, singlesWeightedGameDiff_valid_samples, singlesWinPercentage_valid_samples, singlesOpponentUtr_valid_samples, singlesLongestWinStreak_valid_samples]


doublesGameDiff_valid_samples = []
doublesWeightedGameDiff_valid_samples = []
doublesWinPercentage_valid_samples = []
doublesOpponentUtr_valid_samples = []
doublesLongestWinStreak_valid_samples = []

valid_sets["doubles"] = [doublesGameDiff_valid_samples, doublesWeightedGameDiff_valid_samples, doublesWinPercentage_valid_samples, doublesOpponentUtr_valid_samples, singlesLongestWinStreak_valid_samples]


serveSpeed_valid_labels = []

In [9]:
print(len(data))

76


### Getting Training and Varification sets data from data dictionary

In [ ]:
# Need to do: get all data from data dict into training_labels and training_sameples
# Each stat can go in its individual list, I will just have to be careful that every individual number (Stat) for each player is the same 
# index in each list. For example, Sinner's serve percentage will have the same index as his singles UTR, doubles UTR, etc. 

In [51]:
name_list = list(data)

match_types = ["singles", "doubles"]

# Training Set
for i in range(60):
    training_list = []
    name = random.choice(name_list)
    name_list.remove(name)
    player = data[name]
    # print(player)

    servePercentage_train_samples.append(player[1])
    singlesUtr_train_samples.append(player[2])
    doublesUtr_train_samples.append(player[3])

    for match_type in match_types:  
        dict_string = f"{match_type}"
        match_type_data = player[4][dict_string]
        
        if match_type_data == None:    # for singes and doubles stats, if no activity is recorded, put None for stats variables
            for stats_list in training_sets[dict_string]:
                stats_list.append(None)
        else:
            for i in range(5): 
                training_sets[dict_string][i].append(match_type_data[i])   
        


for i in range(10):
    valid_list = []
    name = random.choice(name_list)
    name_list.remove(name)
    player = data[name]
    # print(player)

    servePercentage_valid_samples.append(player[1])
    singlesUtr_valid_samples.append(player[2])
    doublesUtr_valid_samples.append(player[3])

    for match_type in match_types:  
        dict_string = f"{match_type}"
        match_type_data = player[4][dict_string]
        
        if match_type_data == None:    # for singes and doubles stats, if no activity is recorded, put None for stats variables
            for stats_list in valid_sets[dict_string]:
                stats_list.append(None)
        else:
            for i in range(5): 
                valid_sets[dict_string][i].append(match_type_data[i])              
            
    
    

    
    
# Varification Set
for i in range(10):
    name = random.choice(name_list)
    name_list.remove(name)
    # print(index)

print(f"Players left for test set: {len(name_list)}")


[None, 6.9, 14.68, 15.27, 15.33, 5.5, 8.17, 15.34, 9.75, 15.28, 4.75, 15.33, 15.2, 4.7, 4.21, 15.27, 5.73, 15.49, 15.34, 14.94, 14.96, 6.61, 2.17, 15.19, 8.14, 15.18, 15.43, 15.34, 15.07, 3.17, 7.36, 15.35, 15.34, 5.5, 15.24, 15.23, 4.2, 8.43, 15.35, 15.12, 14.99, 15.09, 15.35, 4.46, 15.17, 14.75, 15.32, 5.9, 15.22, 14.99, 14.74, 14.93, 14.93, 15.3, 7.32, 15.01, 4.9, 6.0, 15.29, 15.37, 15.23, 15.33, 15.28, 7.74, 6.61, 15.29, 15.36, 15.22, 15.28, 14.93, 14.68, 6.0, 6.53, None, 15.51, 15.17, 14.99, 15.18, 8.14, 4.75, 15.34, 15.37, 15.09, 15.01, 15.33, 3.17, 15.34, 8.17, 15.3, 4.2, 15.27, 7.5, 14.96, 4.9, 15.43, 14.99, 14.94, 15.19, 4.46, 15.35, 5.9, 9.75, None, 15.1, 15.2, 15.5, 5.5, 7.36, 15.49, 15.15, 15.38, 15.24, 15.34, 2.17, 15.09, 8.43, 5.73, 14.74, 15.12, 15.34, 7.5, 15.34, 15.28, 15.3, 4.75, 15.37, 14.93, 5.5, 15.09, 14.68, 5.9, 4.46, 14.93, 15.28, 14.96, 15.22, 15.15, 6.61, 15.27, 15.29, 14.75, 15.34, 15.33, 15.19, 15.36, 14.99, 7.74, 15.3, 7.36, 15.38, 14.74, None, 15.09, 15.35

In [55]:
print(f"elements in data: {len(data)}")
print(f"Elements left to use for test set: {len(name_list)}")

elements in data: 76
Elements left to use for test set: 6
